In [78]:
#!pip install -q ipython-autotime
%load_ext autotime
%load_ext autoreload
%timeit
%autoreload 2

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
time: 156 ms (started: 2023-07-25 10:54:53 +03:00)


In [79]:
import pandas as pd
import numpy as np
import torch 
from torch import nn
from Utils import *
from preprocess import *
from models import ANN, CNN1D, BILSTM
from torch.utils.data import Dataset, DataLoader



time: 125 ms (started: 2023-07-25 10:54:53 +03:00)


In [80]:
loaded_data = np.load("cleaned-dataset\data_npy.npy")
X_train, X_test, y_train, y_test = split_data(loaded_data, ratio=0.70, seed=42)

training_dataset = TextDataset(X_train, y_train)
testing_dataset = TextDataset(X_test, y_test)
train_loader = DataLoader(dataset=training_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(dataset=training_dataset, batch_size=128, shuffle=True)

time: 140 ms (started: 2023-07-25 10:54:53 +03:00)


In [81]:
def acc_func(y_true, y_pred):
    y_pred = torch.round(y_pred)
    return (y_true == y_pred).sum() / len(y_true)

time: 156 ms (started: 2023-07-25 10:54:54 +03:00)


In [82]:
x, y = next(iter(train_loader))
x.shape, x.unsqueeze(1).shape, x.unsqueeze(-1).shape, x.unsqueeze(0).shape

(torch.Size([128, 100]),
 torch.Size([128, 1, 100]),
 torch.Size([128, 100, 1]),
 torch.Size([1, 128, 100]))

time: 110 ms (started: 2023-07-25 10:54:54 +03:00)


In [83]:
INPUT_SIZE = x.shape[1]



time: 125 ms (started: 2023-07-25 10:54:54 +03:00)


In [84]:
#* GLOABL VARIABLES
EPOCHS = 150
LEARNING_RATE = 0.0001
# DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DEVICE = torch.device("cpu")
model = CNN1D().to(DEVICE)

#* LSTM 
INPUT_SIZE = x.shape[1]
HIDDEN_STATE = 64
NUM_LAYERS = 4
NUM_CLASSES = 1 #* binary classification

# model = BILSTM(INPUT_SIZE, HIDDEN_STATE, NUM_LAYERS, NUM_CLASSES, bidirection=True).to(DEVICE)


loss = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)





time: 125 ms (started: 2023-07-25 10:54:54 +03:00)


In [85]:
from torchmetrics import Accuracy
accum_acc = 0
for i in range(10):
    x, y = next(iter(train_loader))
    x = x.to(DEVICE)
    y = y.to(DEVICE)
    model.train()
    preds = model(x).squeeze()
    lossval= loss(preds, y)
    optimizer.zero_grad()
    lossval.backward()
    optimizer.step()
    acc = Accuracy(task='binary').to(DEVICE)
    print(acc(preds, y).item())
    # model.train()
    accum_acc += acc(preds, y).item()

0.5078125
0.46875
0.4921875
0.46875
0.4921875
0.515625
0.5078125
0.4921875
0.5234375
0.53125
time: 250 ms (started: 2023-07-25 10:54:54 +03:00)


In [86]:
# torch.save(ann.state_dict(), "model.pth")
# ann_loaded = ANN().to(DEVICE)
# ann_loaded.load_state_dict(torch.load("model.pth"))

time: 172 ms (started: 2023-07-25 10:54:55 +03:00)


In [90]:
history = Training(
    model,
    train_loader,
    test_loader,
    EPOCHS,
    DEVICE,
    loss,
    optimizer,
    print_every=1,
    load_saved_model=True,
)

Loading saved model
Epoch 0 | Train Loss: 0.27408
Epoch 0 | Train Acc: 83.86%
Epoch 0 | Test Loss: 0.13298
Epoch 0 | Test Acc: 95.21%
Test loss decreased from inf to 0.13298 saving new best model
--------------------------------------------------
Epoch 1 | Train Loss: 0.26658
Epoch 1 | Train Acc: 84.02%
Epoch 1 | Test Loss: 0.13353
Epoch 1 | Test Acc: 95.18%
--------------------------------------------------
Epoch 2 | Train Loss: 0.26785
Epoch 2 | Train Acc: 84.26%
Epoch 2 | Test Loss: 0.13254
Epoch 2 | Test Acc: 95.09%
Test loss decreased from 0.13298 to 0.13254 saving new best model
--------------------------------------------------
Epoch 3 | Train Loss: 0.26810
Epoch 3 | Train Acc: 84.19%
Epoch 3 | Test Loss: 0.13205
Epoch 3 | Test Acc: 95.13%
Test loss decreased from 0.13254 to 0.13205 saving new best model
--------------------------------------------------
Epoch 4 | Train Loss: 0.26641
Epoch 4 | Train Acc: 84.34%
Epoch 4 | Test Loss: 0.13080
Epoch 4 | Test Acc: 95.22%
Test loss de

In [ ]:
# loaded_ann = ANN().to(DEVICE)
# loaded_ann.load_state_dict(torch.load("ANN_best_model.pth"))

<All keys matched successfully>

time: 140 ms (started: 2023-07-24 21:23:44 +03:00)


In [ ]:
sample = "The billionaire changed his profile picture to the new logo and added X.com to his Twitter bio."
path = "C:\\Users\\DELL\\OneDrive\\Desktop\\AI projects\\Fake-News-Detection\\word2vec_model\\word2vec.model"
word2vec_model = Load_word2vec(path)

from preprocess import *
predict(sample, word2vec_model, model, DEVICE)
# sample = preprocess_text(sample)
# sample = vectorize_text(sample, word2vec_model)
# model(torch.tensor(sample).float().to(DEVICE))
# print(predict(sample, word2vec_model, loaded_ann, DEVICE))


Word2Vec model loaded successfully!


'Real, with probability: 88.13%'

time: 954 ms (started: 2023-07-25 10:54:25 +03:00)


In [ ]:
# Importing metrics
from sklearn.metrics import accuracy_score,confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt


# Prediction
y_pred = ann(torch.tensor(X_test).float().to(DEVICE)).to("cpu").detach().numpy().reshape(-1)


# Accuracy score
print("Model accuracy is ",accuracy_score(y_pred,y_test))

confusion_matrix = confusion_matrix(y_pred=y_pred,y_true=y_test)

fig,ax = plt.subplots(figsize=(6,6))
sns.heatmap(confusion_matrix,annot=True,fmt="0.1f",linewidths=1.5)
plt.show()